# EECE 5554 Lab 1: A Little about ROSbags


This Jupyter Notebook goes through some of the features you'll need to know to work with ROSbags. It will cover: 
- Recording to a ROSbag
- Getting information about a ROSbag
- Reading in ROSbags to store as other file types

Of course, the documentation is always the best thing to read first.

Specifically for Lab 1, you will want to check out: <br>
Recording and playing back data tutorial: 
https://docs.ros.org/en/jazzy/Tutorials/Beginner-CLI-Tools/Recording-And-Playing-Back-Data/Recording-And-Playing-Back-Data.html <br>
Reading messages from bagfile tutorial: https://docs.ros.org/en/jazzy/Tutorials/Advanced/Recording-A-Bag-From-Your-Own-Node-Py.html


### Recording to/ writing to a ROSbag

There are multiple ways you can write to a ROS bag, but we recommend this way: 
- Using  <code>ros2 bag record  /topic_1 /topic_2 … /topic_n</code> from the command line, which will record the topic(s) you specify. The documentation has more information on relevant command line options.

A few examples of rosbag record: 
| <code>command</code> | What it does  |
| :---- | :---- | 
| <code>ros2 bag record --duration=30 /chatter </code> | Record for 30 s on topic /chatter | 
| <code>ros2 bag record --split --size=1024 /chatter </code> | Record and split bags into smaller sizes of 1024 MB | 
| <code>ros2 bag record -a </code> | Record all topics | 
| <code>ros2 bag record -o /mnt/pen/ /chatter </code> | Record timestamp-named bag on topic /chatter to directory /mnt/pen (AKA, a flash drive) | 

- In a Python script using <code>writer.write('topic_name', serialized_msg, timestamp)</code>.

### Get information about a ROS bag

<code>ros2 bag info bag_name</code> will tell you lots of useful info about the date the bag was collected, its size and number of messages, topic names, etc.

### Reading rosbags to process data or convert to other filetypes (csv, etc.)
- Using a Python script to read in the bag, then store the message in a CSV
- Using <code>ros2 topic echo -b file.bag -p /topic</code><br>
This approach is pretty slow and can only read one topic at once, so not recommended for large files or multiple topics.

The following example is a Python script that uses the `mcap_ros2` package to read ROS2 bag files (in MCAP format) and export them to CSV. 

First, install the mcap-ros2-support package:
```bash
pip install mcap-ros2-support
```

Below is a minimal example using <code>mcap_ros2</code> to read a bag and export to CSV:

In [ ]:

from mcap_ros2.reader import read_ros2_messages
import csv

bag_path = 'my_bag/my_bag_0.mcap'  # Replace with your bag file path
topic_name = '/gps'  # Replace with your topic name
output_file = 'output.csv'

with open(output_file, 'w', newline='') as csvfile:
    writer = None
    count = 0
    
    for msg in read_ros2_messages(bag_path):
        if msg.channel.topic == topic_name:
            ros_msg = msg.ros_msg
            
            # Print fields from first message only (helpful for debugging)
            if count == 0:
                print("Message fields:")
                for field in dir(ros_msg):
                    if not field.startswith('_'):
                        print(f"  {field}: {getattr(ros_msg, field, 'N/A')}")
                print("-" * 50)
            
            # Adjust these fields to match your message type
            row = {
                'timestamp': msg.log_time,
                'latitude': ros_msg.latitude,
                'longitude': ros_msg.longitude,
                'altitude': ros_msg.altitude,
                'utm_easting': ros_msg.utm_easting,
                'utm_northing': ros_msg.utm_northing,
            }
            
            if writer is None:
                writer = csv.DictWriter(csvfile, fieldnames=row.keys())
                writer.writeheader()
            
            writer.writerow(row)
            count += 1

print(f'\nDone! Saved {count} messages to {output_file}')

### The rosbag cookbook has several great examples of how to use the Python API for rosbags, like: 
- Timestamps and metadata
- Cropping/splitting bags (in the case of a very large bag)
- Exporting to CSV <br>
Check it out! https://docs.ros.org/en/rolling/p/rosbag2_py 
